In [1]:
import numpy as np
import matplotlib as plt
import h5py
import scipy
from PIL import Image

/Users/rutvikshah/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
def load_dataset():
    train_dataset = h5py.File('datasets/train_catvnoncat.h5', "r")
    train_set_x_orig = np.array(train_dataset["train_set_x"][:]) # your train set features
    train_set_y_orig = np.array(train_dataset["train_set_y"][:]) # your train set labels

    test_dataset = h5py.File('datasets/test_catvnoncat.h5', "r")
    test_set_x_orig = np.array(test_dataset["test_set_x"][:]) # your test set features
    test_set_y_orig = np.array(test_dataset["test_set_y"][:]) # your test set labels

    classes = np.array(test_dataset["list_classes"][:]) # the list of classes

    train_set_y_orig = train_set_y_orig.reshape((1, train_set_y_orig.shape[0]))
    test_set_y_orig = test_set_y_orig.reshape((1, test_set_y_orig.shape[0]))

    return train_set_x_orig, train_set_y_orig, test_set_x_orig, test_set_y_orig, classes



In [3]:
train_set_x_orig, train_set_y, test_set_x_orig, test_set_y, classes = load_dataset()

In [4]:
m_train = len(train_set_x_orig)
m_test  = len(test_set_x_orig)
h = train_set_x_orig[0].shape[0]
w = train_set_x_orig[0].shape[1]
print("Number of Training Examples : ",m_train)
print("Number of Test     Examples : ",m_test)
print("Height of Image in pixels   : ",h)
print("Width  of Image in pixels   : ",w)

Number of Training Examples :  209
Number of Test     Examples :  50
Height of Image in pixels   :  64
Width  of Image in pixels   :  64


In [5]:
train_set_x_flatten = train_set_x_orig.reshape(train_set_x_orig.shape[0], -1).T
test_set_x_flatten = test_set_x_orig.reshape(test_set_x_orig.shape[0], -1).T
print ("train_set_x_flatten shape: " + str(train_set_x_flatten.shape))
print ("train_set_y shape: " + str(train_set_y.shape))
print ("test_set_x_flatten shape: " + str(test_set_x_flatten.shape))
print ("test_set_y shape: " + str(test_set_y.shape))
print ("sanity check after reshaping: " + str(train_set_x_flatten[0:5,0]))

train_set_x_flatten shape: (12288, 209)
train_set_y shape: (1, 209)
test_set_x_flatten shape: (12288, 50)
test_set_y shape: (1, 50)
sanity check after reshaping: [17 31 56 22 33]


In [6]:
train_set_x = train_set_x_flatten/255
test_set_x  = test_set_x_flatten/255

In [7]:
def sigmoid(z):
    sig = 1/(1 + np.exp(-z))
    return sig

In [8]:
def propagate(w, b, X, Y):
    m = X.shape[1]
    Y1 = sigmoid(np.dot(w.T,X) + b)
    cost = -(1/m)*( np.sum( (Y*np.log(Y1)) + ((1-Y)*np.log(1-Y1)) ) )
    dw = (1/m)*( np.dot(X,(Y1-Y).T) )
    db = (1/m)*( np.sum(Y1-Y) )

    cost = np.squeeze(cost)
    grads = {"dw":dw, "db":db}

    return grads, cost

In [9]:
def optimize(w, b, X, Y, num_iterations, learning_rate, print_cost = False):
    costs = []
    for i in range(num_iterations):
        grads, cost = propagate(w, b, X, Y)
        dw = grads["dw"]
        db = grads["db"]

        w = w - learning_rate*dw
        b = b - learning_rate*db

        if i%100 == 0:
            costs.append(cost)

        if print_cost and i%100 == 0:
            print("Cost after %i iteration is %f" %(i, cost))

        params = {"w":w, "b":b}
        grads  = {"dw":dw, "db":db}

    return params, grads, costs

In [10]:
def predict(w, b, X):
    m = X.shape[1]
    Y_prediction = np.zeros((1,m))
    w = w.reshape(X.shape[0], 1)

    Y1 = sigmoid(np.dot(w.T,X) + b)

    for i in range(Y1.shape[1]):
        if(Y1[0,i] > 0.5):
            Y_prediction[0,i] = 1
        else:
            Y_prediction[0,i] = 0
    return Y_prediction

In [11]:
def model(X_train, Y_train, X_test, Y_test, num_iterations = 2000, learning_rate = 0.225, print_cost = False):
    w,b = np.zeros((X_train.shape[0],1)), 0
    params, grads, costs = optimize(w, b, X_train, Y_train, num_iterations, learning_rate, print_cost)

    w = params["w"]
    b = params["b"]

    Y_prediction_test = predict(w, b, X_test)
    Y_prediction_train = predict(w, b, X_train)

    print("train accuracy: {} %".format(100 - np.mean(np.abs(Y_prediction_train - Y_train)) * 100))
    print("test accuracy: {} %".format(100 - np.mean(np.abs(Y_prediction_test - Y_test)) * 100))


    d = {"costs": costs,
         "Y_prediction_test": Y_prediction_test,
         "Y_prediction_train" : Y_prediction_train,
         "w" : w,
         "b" : b,
         "learning_rate" : learning_rate,
         "num_iterations": num_iterations}

    return d

In [12]:
d = model(train_set_x, train_set_y, test_set_x, test_set_y, num_iterations = 2000, learning_rate = 0.005, print_cost = True)

Cost after 0 iteration is 0.693147
Cost after 100 iteration is 0.584508
Cost after 200 iteration is 0.466949
Cost after 300 iteration is 0.376007
Cost after 400 iteration is 0.331463
Cost after 500 iteration is 0.303273
Cost after 600 iteration is 0.279880
Cost after 700 iteration is 0.260042
Cost after 800 iteration is 0.242941
Cost after 900 iteration is 0.228004
Cost after 1000 iteration is 0.214820
Cost after 1100 iteration is 0.203078
Cost after 1200 iteration is 0.192544
Cost after 1300 iteration is 0.183033
Cost after 1400 iteration is 0.174399
Cost after 1500 iteration is 0.166521
Cost after 1600 iteration is 0.159305
Cost after 1700 iteration is 0.152667
Cost after 1800 iteration is 0.146542
Cost after 1900 iteration is 0.140872
train accuracy: 99.04306220095694 %
test accuracy: 70.0 %


In [13]:
learning_rates = [0.25, 0.35, 0.45]
models = {}
for i in learning_rates:
    print ("learning rate is: " + str(i))
    models[str(i)] = model(train_set_x, train_set_y, test_set_x, test_set_y, num_iterations = 1500, learning_rate = i, print_cost = False)
    print ('\n' + "-------------------------------------------------------" + '\n')

learning rate is: 0.25


/Users/rutvikshah/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:4: RuntimeWarning: divide by zero encountered in log
  after removing the cwd from sys.path.
/Users/rutvikshah/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:4: RuntimeWarning: invalid value encountered in multiply
  after removing the cwd from sys.path.


train accuracy: 100.0 %
test accuracy: 72.0 %

-------------------------------------------------------

learning rate is: 0.35
train accuracy: 100.0 %
test accuracy: 74.0 %

-------------------------------------------------------

learning rate is: 0.45
train accuracy: 100.0 %
test accuracy: 74.0 %

-------------------------------------------------------

